<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2024/images/logo.png?raw=true" alt="2023年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

# ゲノムアセンブル
　近年、シロイヌナズナやイネなどのようなモデル生物だけでなく、さまざまな生物でゲノム配列の整備（ゲノム解読）が進められています。解読されたゲノム配列は、進化学的研究や育種学的研究など、幅広い研究の基盤情報になります。

　次世代シーケンシング（ショートリードシーケンシング）や第三世代シーケンシング（ロングリードシーケンシング）をおこなうと、元々のゲノム配列に由来する、短い塩基配列（**リード、Read**）を多数得られます。ゲノム解読は、多数の短い配列をつなぎ、元のゲノム配列を復元する試みです。その復元処理は**ゲノムアセンブル（Genome assemble）**と呼ばれています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/assemble_overview.png?raw=true" height="100px" align="middle">

　今回の実習では、ショートリードやロングリードからゲノム配列を復元できるのかを見ていきます。また、ショートリードやロングリードの特徴についても勉強していきます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_and_long_reads.png?raw=true" height="100px" align="middle">



### サンプルデータのダウンロード
　この実習で使用するサンプルデータをダウンロードします。

　このテキストでは、約40 kbのゲノム配列から仮想的に得たリードデータをサンプルデータとして使用します。


In [ ]:
%%bash
wget -q -O 'assemble_dataset.sh' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/scripts/assemble_dataset.sh
bash assemble_dataset.sh
ls | grep -e 'fastq'

### ソフトウェアのダウンロード

　今回使用するソフトウェアとその実行プログラムをダウンロードをします。



In [ ]:
%%bash
wget -q -O 'assemble_softwares.sh' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/scripts/assemble_softwares.sh
bash assemble_softwares.sh
ls tools/

### 今回の勉強内容

1. ショートリードを使ったゲノムアセンブル
 1. ショートリードを確認する
 1. ゲノムアセンブル
 1. ゲノム配列を確認する

1. ロングリードを使ったゲノムアセンブル
 1. ロングリードを確認する
 1. ゲノムアセンブル
 1. ゲノム配列を確認する

1. ゲノム配列を改良する






---
## 1. ショートリードを使ったゲノムアセンブル

　ショートリードのゲノムアセンブルをおこない、ゲノムの再構築を試みてみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_read_assemble.png?raw=true" height="100px" align="middle">

### 1-1. ショートリードを確認する
　最初に、ゲノムアセンブルに使用するショートリードデータを確認してみましょう。DNA断片の両端をシーケンシングした**ペアエンド（Paired-end）**という形のリードデータを使用します。ペアエンドのデータは、通常2つのファイルに分かれており、それらを1組のデータとして同時に扱います。
- Read1: /content/illumina_read1.fastq
- Read2: /content/illumina_read2.fastq

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/paired_end_reads.png?raw=true" height="200px" align="middle">

#### リードのデータファイル形式（FASTQ）

　リードのデータは**FASTQ**と呼ばれる形式でファイルに保存されています。FASTQ形式では、4行で1つのリードデータの情報を表しています。

```
@リードの名前
TGAAGCAGT..............(リードの塩基配列)
+
F1GCFCGGE..............(各塩基のクオリティ)
```

- 第1行（@からはじまる行）: リードの名前
- 第2行: リードの塩基配列
- 第3行（+からはじまる行）: 第2行と第4行の区切りのための行
- 第4行: 各塩基配列のクオリティ（ASCIIコードで書かれている）。1文字が何らかのクオリティスコア（数値）を示している。
 - 参考: https://bi.biopapyrus.jp/rnaseq/qc/fastq-quality-score.html



#### seqkitを使ったリードの統計情報確認

 　[seqkit](https://bioinf.shenwei.me/seqkit/)というソフトウェアを使って、ペアエンドのうちRead1の統計情報（リードの数やリードの長さなどの情報）を確認してみましょう。

```sh
# リードの統計情報の確認
seqkit stat 塩基配列データ
```

In [ ]:
%%bash
seqkit stat illumina_read1.fastq

### 実習1

　seqkitを使って、もう一方のリード（ペアエンドのRead2: illumina_read2.fastq）の統計情報を確認してください。

In [ ]:
%%bash
seqkit stat

### 1-2. ゲノムアセンブル

　[Platanus](http://platanus.bio.titech.ac.jp/)というショートリード用のゲノムアセンブルソフトウェアを使って、ペアエンドのショートリードからゲノム配列の復元を試みてみましょう。今回は、事前に用意したゲノムアセンブル用プログラムで、ペアエンドショートリードのゲノムアセンブルをおこないます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/genome_assemble.png?raw=true" height="250px" align="middle">

In [ ]:
%%bash
bash Platanus_assemble.sh illumina_read1.fastq illumina_read2.fastq

### 1-3. ゲノム配列を確認する

　アセンブルで得られたゲノム配列（**アセンブリ、Assembly**）を確認してみましょう。Colabサーバー上の次のファイルに保存されています。

/content/platanus_output/illumina_contig.fa

得られたゲノム配列は、1本ではなく、複数本の配列に分かれており、元のゲノム配列を完全に再現できていません。アセンブル後の複数本に分かれた配列は**コンティグ（Contig）**と呼ばれています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/contigs.png?raw=true" height="100px" align="middle">

　seqkitを使って、コンティグの統計情報（配列数、配列の平均長、N50など）をみてみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/about_n50.png?raw=true" height="100px" align="middle">


In [ ]:
%%bash
## オプション「-a」を追加すると、N50を含む情報を得られる
seqkit stat -a /content/platanus_output/illumina_contig.fa

　次世代シーケンシングによるリードには、低い割合ですが塩基の読み間違い（エラー）が含まれています。そのエラーがアセンブルに反映されていないか（コンティグの各塩基の精度）を目で確認してみましょう。

　アセンブルに使ったショートリードを、アセンブル後のコンティグに並べて（**アライメント、Alignment**して）、そのアライメント結果を[IGV](https://igv.org/doc/desktop/)と呼ばれるツールで確認します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_read_alignment.png?raw=true" height="250px" align="middle">

In [ ]:
%%bash
## アセンブル後のコンティグにリードをアライメントする
CONTIG="/content/platanus_output/illumina_contig.fa"
OUTPUT="illumina_alignment"
READ1="illumina_read1.fastq"
READ2="illumina_read2.fastq"

bash Alignment.sh $CONTIG $OUTPUT $READ1 $READ2

In [ ]:
## アライメント結果をみる
ctg = "/content/platanus_output/illumina_contig.fa"
aln = "illumina_alignment.bam"

## ---- 以下、変更不要 -----
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
ref = RefTrack({ "fastaPath": ctg, "indexPath": ctg+".fai", "id": ctg })
bam = BamTrack({ "name": aln, "path": aln, "viewAsPairs":True })
b = igv_notebook.Browser(ref)
b.load_track(bam)

### ショートリードゲノムアセンブルのまとめ

　ショートリードシーケンシングは、非常に多数の短い配列（リード）を得られます。塩基の読み間違いも少ない（シーケンシングエラー率も低い）です。

　生物のゲノムは、同じ塩基パターンの繰り返し（リピート）などを多く含んでいます。ショートリードは、リード長よりも長いリピート配列に弱く、ショートリードだけでゲノムを再現するのはほぼ不可能です。一方で、ショートリードは、シーケンシングエラーが少ないため、精度の高いコンティグが得られます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_reads_and_repeat.png?raw=true" height="250px" align="middle">

---
## 2. ロングリードを使ったゲノムアセンブル

　ショートリードシーケンシングは、50-300 bpの短い塩基配列を多量に得るシーケンシングです。近年、非常に長い塩基配列を得るシーケンシング技術（**ロングリードシーケンシング、Long read sequencing**）が発達してきました。Oxford Nanopore Technologies社のナノポアシーケンシングや、Pacific Biosciences社のPacBioシーケンシングがそれにあたります。

　現在では、ロングリードを使ったゲノムアセンブルが最もよくおこなわれています。ここでは、ナノポアシーケンシングによるロングリードからどのようなゲノム配列を復元できるかを見ていきましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/long_read_assemble.png?raw=true" height="100px" align="middle">

### 2-1. ロングリードを確認する

　ロングリードのシーケンスデータを見てみましょう。

- ロングリード: /content/nanopore.fastq

　seqkitを使って、ロングリードの統計情報も確認してみましょう。

In [ ]:
%%bash
seqkit stat nanopore.fastq

### 2-2. ゲノムアセンブル
　[Flye](https://github.com/fenderglass/Flye)というソフトウェアを使って、ロングリードのゲノムアセンブルをおこなってみましょう。




In [ ]:
%%bash
bash Flye_assemble.sh nanopore.fastq

### 2-3. ゲノム配列を確認する

　アセンブルで得られたゲノム配列は、次のファイルに保存されています。

/content/flye_output/assembly.fasta

　seqkitで、アセンブルで得られたコンティグ数や長さ、N50などの統計情報を確認してみましょう。

In [ ]:
%%bash
seqkit stat /content/flye_output/assembly.fasta

　コンティグに対してロングリードをアライメントし、そのアライメント結果をIGVで確認してみましょう。

In [ ]:
%%bash
## アセンブル後のコンティグにリードをアライメントする
CONTIG="/content/flye_output/assembly.fasta"
OUTPUT="nanopore_alignment"
READ="nanopore.fastq"

bash Alignment.sh $CONTIG $OUTPUT $READ

In [ ]:
## アライメント結果をみる
ctg = "/content/flye_output/assembly.fasta"
aln = "nanopore_alignment.bam"

## ---- 以下、変更不要 -----
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
ref = RefTrack({ "fastaPath": ctg, "indexPath": ctg+".fai", "id": ctg })
bam = BamTrack({ "name": aln, "path": aln, "viewAsPairs":False })
b = igv_notebook.Browser(ref)
b.load_track(bam)

### 実習2

　ロングリードで作ったコンティグに対して、ショートリードをアライメントし、そのアライメント結果を確認してください。

- 5行目にREAD1のデータファイル（`illumina_read1.fastq`）、7行目にREAD2のデータファイル（`illumina_read2.fastq`）を入力し、実行してください。

In [ ]:
## アセンブル後のコンティグにリードをアライメントする
CONTIG="/content/flye_output/assembly.fasta"
OUTPUT="nanopore_short_read_alignment"
## ショートリードのREAD1
READ1=""
## ショートリードのREAD2
READ2=""

## ---- 以下、変更不要 -----
## アライメントの実行
!bash Alignment.sh $CONTIG $OUTPUT $READ1 $READ2

## アライメント結果の確認
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
ref = RefTrack({ "fastaPath": CONTIG, "indexPath": CONTIG+".fai", "id": CONTIG })
bam = BamTrack({ "name": OUTPUT+".bam", "path": OUTPUT+".bam", "viewAsPairs":True })
b = igv_notebook.Browser(ref)
b.load_track(bam)

### ロングリードゲノムアセンブリのまとめ

　ロングリードは、シーケンシングエラー率が高く、構築されたコンティグの塩基が正確ではない場合があります。しかし、ショートリードの場合とは異なり、ロングリードを使ったゲノムアセンブリは、非常に長いコンティグを生成できます。そのため、リピート配列を多く含む生物のゲノムアセンブリにおいて、ロングリードは主力的な役割を果たしています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/long_read_assemble2.png?raw=true" height="250px" align="middle">

---
## 3. ゲノム配列を改良する

　リード数が多く、塩基情報が正確なショートリードと、長いコンティグを生成できるロングリードを組み合わせることで、より良いゲノム配列を得ることができます。

　先ほど得られたロングリードのゲノム配列（コンティグ）には、塩基情報が不正確そうな箇所がみられます。ここでは、ショートリードでその不正確性の補正を試みてみましょう。この補正処理は **ポリッシング（Polishing）** と呼ばれています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/polishing.png?raw=true" height="220px" align="middle">

### ショートリードを使った補正

In [ ]:
%%bash
CONTIG="/content/flye_output/assembly.fasta"
OUTPUT="assembly"
READ1="illumina_read1.fastq"
READ2="illumina_read2.fastq"

bash Polishing.sh $CONTIG $OUTPUT $READ1 $READ2

### 補正後のゲノム配列を確認する

　補正後のゲノム配列にショートリードをアライメントし、その結果をIGVで確認してみましょう。

In [ ]:
## アセンブル後のコンティグにリードをアライメントする
CONTIG="/content/assembly_polished.fasta"
OUTPUT="illumina_reads_to_polished_assembly"
READ1="illumina_read1.fastq" ## ショートリードのREAD1
READ2="illumina_read2.fastq" ## ショートリードのREAD2

## ---- 以下、変更不要 -----
## アライメントの実行
!bash Alignment.sh $CONTIG $OUTPUT $READ1 $READ2

## アライメント結果の確認
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
ref = RefTrack({ "fastaPath": CONTIG, "indexPath": CONTIG+".fai", "id": CONTIG })
bam = BamTrack({ "name": OUTPUT+".bam", "path": OUTPUT+".bam", "viewAsPairs":True })
b = igv_notebook.Browser(ref)
b.load_track(bam)

---
## まとめ

　今回、ショートリードとロングリードを使ってゲノムアセンブルをおこない、ゲノム配列の復元を試みました。

　ショートリードでは、正確なコンティグを得られる一方で、ゲノム全体を再現するほど充分長い配列は得られませんでした。要因としては、ゲノムに含まれるリピート配列の存在などが考えられます。

　ロングリードでは、非常に長いコンティグを得ることができました。しかし、エラー率が高いリードを使ったため、部分的には塩基情報が不正確なコンティグが生成されました。

　ショートリードとロングリードそれぞれの利点を活かすことで、より良いゲノム配列を構築することができます。今回の実習でも、ロングリードで得られた長いコンティグを、ショートリードの正確性で補い、高精度なゲノム配列を得ることができました。この手順は、実際のゲノム解読でもよく使われています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/major_assemble_proc.png?raw=true" height="220px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>